In [2]:
import sagemaker
import boto3
from sagemaker.local import LocalSession

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-mnist"

iam = boto3.client('iam')
sagemaker_role = iam.get_role(RoleName='SagemakerRole')['Role']['Arn']

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Dmitry\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Dmitry\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = [
    f"https://sagemaker-example-files-prod-{region}.s3.amazonaws.com/datasets/image/MNIST/"
]

MNIST(
    "data",
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    ),
)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [4]:
inputs = sagemaker_session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-514157727917/sagemaker/DEMO-pytorch-mnist


In [6]:
!pygmentize mnist.py

import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 32

In [7]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="mnist.py",
    role=sagemaker_role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=2,
    instance_type="ml.c5.2xlarge",
    hyperparameters={"epochs": 1, "backend": "gloo"},
)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Dmitry\AppData\Local\sagemaker\sagemaker\config.yaml


In [8]:
estimator.fit({"training": inputs})

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-09-20-12-53-41-160


2023-09-20 12:53:45 Starting - Starting the training job...
2023-09-20 12:54:00 Starting - Preparing the instances for training......
2023-09-20 12:55:03 Downloading - Downloading input data...
2023-09-20 12:55:33 Training - Downloading the training image...
2023-09-20 12:56:19 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-09-20 08:56:28,937 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-09-20 08:56:28,938 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-09-20 08:56:28,940 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-09-20 08:56:28,950 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-09-20 08:56:28,952 sagemaker_pytorch_container.training INFO     Invoking user training s

UnexpectedStatusException: Error for Training job pytorch-training-2023-09-20-12-53-41-160: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "raise RuntimeError("Dataset not found. You can use download=True to download it")
 RuntimeError: Dataset not found. You can use download=True to download it"
Command "/opt/conda/bin/python3.8 mnist.py --backend gloo --epochs 1", exit code: 1